In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    check_crs,
    save_parquet_excel
)

In [3]:
data_path = get_data_diretorio()

# Unidades de Conservação

Uma das camadas que poderia ser útil para as áreas verdes públicas é a dos Parques e Unidades de Conservação, que deveria estar dispoível no geosampa. Metadados disponíveis em: https://metadados.geosampa.prefeitura.sp.gov.br/geonetwork/srv/por/catalog.search;jsessionid=8F275348DD1C96B9192D87B0FB8EE8B7#/metadata/48f702b4-f2ea-42f4-bacd-5682ae1731e2

Contudo esta camada está temporariamente indisponível devido a alteração de valores que geravam-na. Após esta alteração ela passou a dar problema e ficou indispomível.

Um dos nossos colaboradores, Elias Noda, já havia feito análises com esta camada anteriomente, e por isso temos acesso a ela. Contudo é uma versão desatualizada e que não pode ser baixada por todos, já que não está mais acessível.

# Importar dados

In [4]:
gdf_ucs = gpd.read_parquet(
    os.path.join(
        data_path,
        'Unidades_de_Conservacao.parquet'
    )
)

In [5]:
gdf_ucs.sample(2)

,id,cd_identificador,nm_unidade_conservacao,cd_cadparc,tx_tipo_situacao_projeto,tx_tipo_categoria_parque_unidade_conservacao,tx_tipo_classificacao_parque_unidade_conservacao,cd_tipo_situacao_projeto,cd_tipo_categoria_parque_unidade_conservacao,cd_tipo_classificacao_parque_unidade_conservacao,...,tx_tipo_categoria,tx_data_inauguracao,tx_instrumento_legal_criacao,tx_situacao,nm_administrador,qt_area_metro,cd_pde_2022,cd_identificador_cadparcs_area_protecao_apa,nm_esfera_administrativa,nm_area_protecao_ambiental
181,cadparcs_parque_unidade_conservacao.227,NaN,None,None,None,None,None,NaN,NaN,NaN,...,Parque Natural,2020,Dec. 43.329/03 e 50.201/08,Existente,SVMA/CGPABI/DGUC,5375725.0,PQ_IQ_05.1,NaN,None,None
251,cadparcs_parque_unidade_conservacao.273,NaN,None,None,None,None,None,NaN,NaN,NaN,...,Parque Estadual - UC,1962,Lei Est. 6.884/62 e Dec. Est. 41.626/63,Existente,SEMIL/FF,42277824.0,None,NaN,None,None


# Padronizaçã de colunas

Como este arquivo não segue os padrões dos outros, vou conferir individualmente as colunas para encontrar as que fazem sentido manter

## Conferir colunas

### id

In [6]:
gdf_ucs['id'].is_unique

True

In [7]:
sum(gdf_ucs['id'].isnull())

0

### cd_identificador

In [8]:
gdf_ucs['cd_identificador'].is_unique

False

In [9]:
sum(gdf_ucs['cd_identificador'].isnull())

23

In [10]:
gdf_ucs['cd_identificador'].unique

<bound method Series.unique of 0      101.0
2      104.0
3      103.0
4      102.0
44       NaN
94       NaN
162      NaN
251      NaN
353      NaN
17       NaN
75       NaN
83       NaN
167      NaN
181      NaN
318      NaN
134      NaN
328      NaN
312      NaN
253      NaN
266      NaN
267      NaN
351      NaN
0        NaN
1        NaN
2        NaN
3        NaN
5        NaN
Name: cd_identificador, dtype: float64>

### nm_unidade_conservacao

In [11]:
gdf_ucs['nm_unidade_conservacao'].is_unique

False

In [12]:
gdf_ucs['nm_unidade_conservacao'].unique

<bound method Series.unique of 0          Anhanguera
2         Mata Virgem
3      Sítio Curucutu
4             Mutinga
44               None
94               None
162              None
251              None
353              None
17               None
75               None
83               None
167              None
181              None
318              None
134              None
328              None
312              None
253              None
266              None
267              None
351              None
0                None
1                None
2                None
3                None
5                None
Name: nm_unidade_conservacao, dtype: object>

### cd_cadparc

In [13]:
gdf_ucs['cd_cadparc'].is_unique

False

In [14]:
gdf_ucs['cd_cadparc'].unique

<bound method Series.unique of 0      UC_PR_01
2      UC_PA_05
3      UC_PA_06
4      UC_PJ_01
44         None
94         None
162        None
251        None
353        None
17         None
75         None
83         None
167        None
181        None
318        None
134        None
328        None
312        None
253        None
266        None
267        None
351        None
0          None
1          None
2          None
3          None
5          None
Name: cd_cadparc, dtype: object>

### tx_tipo_situacao_projeto

Saber se o projeto é existente ou não pode ser um filtro interessante. Contudo não há como ter certeza de que os valores null correspondam a "não existente"

In [15]:
gdf_ucs['tx_tipo_situacao_projeto'].unique

<bound method Series.unique of 0      Existente
2      Existente
3      Existente
4      Existente
44          None
94          None
162         None
251         None
353         None
17          None
75          None
83          None
167         None
181         None
318         None
134         None
328         None
312         None
253         None
266         None
267         None
351         None
0           None
1           None
2           None
3           None
5           None
Name: tx_tipo_situacao_projeto, dtype: object>

### tx_tipo_categoria_parque_unidade_conservacao

In [16]:
gdf_ucs['tx_tipo_categoria_parque_unidade_conservacao'].unique

<bound method Series.unique of 0                     Refúgio de Vida Silvestre
2      Reserva Particular do Patrimônio Natural
3      Reserva Particular do Patrimônio Natural
4      Reserva Particular do Patrimônio Natural
44                                         None
94                                         None
162                                        None
251                                        None
353                                        None
17                                         None
75                                         None
83                                         None
167                                        None
181                                        None
318                                        None
134                                        None
328                                        None
312                                        None
253                                        None
266                                        None
267      

### tx_tipo_classificacao_parque_unidade_conservacao

Aparentemente idem ao anterior, portanto será apagado

In [17]:
gdf_ucs['tx_tipo_classificacao_parque_unidade_conservacao'].unique

<bound method Series.unique of 0                     Refúgio de Vida Silvestre
2      Reserva Particular do Patrimônio Natural
3      Reserva Particular do Patrimônio Natural
4      Reserva Particular do Patrimônio Natural
44                                         None
94                                         None
162                                        None
251                                        None
353                                        None
17                                         None
75                                         None
83                                         None
167                                        None
181                                        None
318                                        None
134                                        None
328                                        None
312                                        None
253                                        None
266                                        None
267      

### cd_tipo_situacao_projeto

In [18]:
gdf_ucs['cd_tipo_situacao_projeto'].unique

<bound method Series.unique of 0      2.0
2      2.0
3      2.0
4      2.0
44     NaN
94     NaN
162    NaN
251    NaN
353    NaN
17     NaN
75     NaN
83     NaN
167    NaN
181    NaN
318    NaN
134    NaN
328    NaN
312    NaN
253    NaN
266    NaN
267    NaN
351    NaN
0      NaN
1      NaN
2      NaN
3      NaN
5      NaN
Name: cd_tipo_situacao_projeto, dtype: float64>

### cd_tipo_categoria_parque_unidade_conservacao

In [19]:
gdf_ucs['cd_tipo_categoria_parque_unidade_conservacao'].unique

<bound method Series.unique of 0      7.0
2      8.0
3      8.0
4      8.0
44     NaN
94     NaN
162    NaN
251    NaN
353    NaN
17     NaN
75     NaN
83     NaN
167    NaN
181    NaN
318    NaN
134    NaN
328    NaN
312    NaN
253    NaN
266    NaN
267    NaN
351    NaN
0      NaN
1      NaN
2      NaN
3      NaN
5      NaN
Name: cd_tipo_categoria_parque_unidade_conservacao, dtype: float64>

### cd_tipo_classificacao_parque_unidade_conservacao

Apesar de no tx aparentar ser identico ao anterior, como os cds estão diferentes, manterei este para comparação posterior

In [20]:
gdf_ucs['cd_tipo_classificacao_parque_unidade_conservacao'].unique

<bound method Series.unique of 0      2.0
2      3.0
3      3.0
4      3.0
44     NaN
94     NaN
162    NaN
251    NaN
353    NaN
17     NaN
75     NaN
83     NaN
167    NaN
181    NaN
318    NaN
134    NaN
328    NaN
312    NaN
253    NaN
266    NaN
267    NaN
351    NaN
0      NaN
1      NaN
2      NaN
3      NaN
5      NaN
Name: cd_tipo_classificacao_parque_unidade_conservacao, dtype: float64>

### cd_identificador_cadparcs_parque_unidade_conservacao

In [21]:
gdf_ucs['cd_identificador_cadparcs_parque_unidade_conservacao'].is_unique

False

In [22]:
gdf_ucs['cd_identificador_cadparcs_parque_unidade_conservacao'].unique

<bound method Series.unique of 0        NaN
2        NaN
3        NaN
4        NaN
44     275.0
94     274.0
162    283.0
251    273.0
353    276.0
17     360.0
75     362.0
83     210.0
167    361.0
181    227.0
318    359.0
134    380.0
328     92.0
312    180.0
253    293.0
266    292.0
267    291.0
351    290.0
0        NaN
1        NaN
2        NaN
3        NaN
5        NaN
Name: cd_identificador_cadparcs_parque_unidade_conservacao, dtype: float64>

### nm_area

In [23]:
gdf_ucs['nm_area'].is_unique

False

In [24]:
gdf_ucs['nm_area'].unique

<bound method Series.unique of 0                                   None
2                                   None
3                                   None
4                                   None
44                    Fontes do Ipiranga
94                          Serra do Mar
162    Horto Florestal - Alberto Lofgren
251                           Cantareira
353                              Jaragua
17                              Varginha
75                             Jaceguava
83              Cabeceiras do Aricanduva
167                                Itaim
181                     Fazenda Do Carmo
318                               Borore
134                   Cratera De Colonia
328                   Cratera De Colonia
312                           Anhanguera
253                          Mata Virgem
266         Solo Sagrado de Guarapiranga
267                       Sitio Curucutu
351                              Mutinga
0                                   None
1                         

### tx_tipo_categoria

In [25]:
gdf_ucs['tx_tipo_categoria'].unique

<bound method Series.unique of 0                           None
2                           None
3                           None
4                           None
44          Parque Estadual - UC
94          Parque Estadual - UC
162         Parque Estadual - UC
251         Parque Estadual - UC
353         Parque Estadual - UC
17                Parque Natural
75                Parque Natural
83                Parque Natural
167               Parque Natural
181               Parque Natural
318               Parque Natural
134               Parque Natural
328               Parque Natural
312    Refugio de Vida Silvestre
253                         RPPN
266                         RPPN
267                         RPPN
351                         RPPN
0                           None
1                           None
2                           None
3                           None
5                           None
Name: tx_tipo_categoria, dtype: object>

### tx_situacao

Acredito que seja uma informação importante, então, apesar de achar que está repetido, irire manter.

In [26]:
gdf_ucs['tx_situacao'].unique

<bound method Series.unique of 0           None
2           None
3           None
4           None
44     Existente
94     Existente
162    Existente
251    Existente
353    Existente
17     Existente
75     Existente
83     Existente
167    Existente
181    Existente
318    Existente
134    Existente
328    Existente
312    Existente
253    Existente
266    Existente
267    Existente
351    Existente
0           None
1           None
2           None
3           None
5           None
Name: tx_situacao, dtype: object>

### qt_area_metro

In [27]:
sum(gdf_ucs['qt_area_metro'].isnull())

4

In [28]:
sum(gdf_ucs['geometry'].isnull())

0

como há 4 valores vazios, prefiro apagar esta coluna e, se necessário, eu crio uma nova a partir da geometria.

### cd_pde_2022

In [29]:
gdf_ucs['cd_pde_2022'].is_unique

False

In [30]:
gdf_ucs['cd_pde_2022'].unique

<bound method Series.unique of 0            None
2            None
3            None
4            None
44           None
94           None
162          None
251          None
353          None
17     PQ_CS_23.1
75     PQ_PA_09.1
83     PQ_SM_03.1
167    PQ_PA_07.2
181    PQ_IQ_05.1
318    PQ_CS_13.1
134    PQ_PA_02.2
328    PQ_PA_02.1
312      UC_PR_01
253          None
266          None
267          None
351          None
0            None
1            None
2            None
3            None
5            None
Name: cd_pde_2022, dtype: object>

### cd_identificador_cadparcs_area_protecao_apa

In [31]:
gdf_ucs['cd_identificador_cadparcs_area_protecao_apa'].is_unique

False

In [32]:
gdf_ucs['cd_identificador_cadparcs_area_protecao_apa'].unique

<bound method Series.unique of 0       NaN
2       NaN
3       NaN
4       NaN
44      NaN
94      NaN
162     NaN
251     NaN
353     NaN
17      NaN
75      NaN
83      NaN
167     NaN
181     NaN
318     NaN
134     NaN
328     NaN
312     NaN
253     NaN
266     NaN
267     NaN
351     NaN
0      35.0
1       1.0
2       3.0
3       2.0
5      34.0
Name: cd_identificador_cadparcs_area_protecao_apa, dtype: float64>

### nm_esfera_administrativa

Acredito que esteja repetida

In [33]:
gdf_ucs['nm_esfera_administrativa'].unique

<bound method Series.unique of 0           None
2           None
3           None
4           None
44          None
94          None
162         None
251         None
353         None
17          None
75          None
83          None
167         None
181         None
318         None
134         None
328         None
312         None
253         None
266         None
267         None
351         None
0      Municipal
1       Estadual
2       Estadual
3       Estadual
5      Municipal
Name: nm_esfera_administrativa, dtype: object>

### nm_area_protecao_ambiental

In [34]:
gdf_ucs['nm_area_protecao_ambiental'].unique

<bound method Series.unique of 0                           None
2                           None
3                           None
4                           None
44                          None
94                          None
162                         None
251                         None
353                         None
17                          None
75                          None
83                          None
167                         None
181                         None
318                         None
134                         None
328                         None
312                         None
253                         None
266                         None
267                         None
351                         None
0                 Capivari-Monos
1                       Iguatemi
2                Varzea do Tiete
3      Parque e Fazenda do Carmo
5                 Borore-Colonia
Name: nm_area_protecao_ambiental, dtype: object>

## Alterar colunas

In [35]:
gdf_ucs.columns

Index(['id', 'cd_identificador', 'nm_unidade_conservacao', 'cd_cadparc',
       'tx_tipo_situacao_projeto',
       'tx_tipo_categoria_parque_unidade_conservacao',
       'tx_tipo_classificacao_parque_unidade_conservacao',
       'cd_tipo_situacao_projeto',
       'cd_tipo_categoria_parque_unidade_conservacao',
       'cd_tipo_classificacao_parque_unidade_conservacao', 'dt_atualizacao',
       'geometry', 'cd_identificador_cadparcs_parque_unidade_conservacao',
       'nm_area', 'tx_tipo_categoria', 'tx_data_inauguracao',
       'tx_instrumento_legal_criacao', 'tx_situacao', 'nm_administrador',
       'qt_area_metro', 'cd_pde_2022',
       'cd_identificador_cadparcs_area_protecao_apa',
       'nm_esfera_administrativa', 'nm_area_protecao_ambiental'],
      dtype='object')

In [36]:
drop_ucs_cols={
    'cd_identificador',
    'cd_cadparc',
    'tx_tipo_classificacao_parque_unidade_conservacao',
    'dt_atualizacao',
    'tx_data_inauguracao',
    'tx_instrumento_legal_criacao',
    'nm_administrador',
    'qt_area_metro'
}

gdf_ucs.drop(
    columns=drop_ucs_cols,
    axis=1,
    inplace=True
)

In [37]:
rename_ucs_cols={
    'id' : 'id_uc',
    'nm_unidade_conservacao' : 'nm_uc',
    'tx_tipo_situacao_projeto' : 'situacao_projeto',
    'tx_tipo_categoria_parque_unidade_conservacao' : 'cat_parq_uc',
    'cd_tipo_situacao_projeto' : 'cd_situacao_projeto',
    'cd_tipo_categoria_parque_unidade_conservacao' : 'cd_cat_parq_uc',
    'cd_tipo_classificacao_parque_unidade_conservacao' : 'cd_clasfc_parq_uc',
    'cd_identificador_cadparcs_parque_unidade_conservacao': 'cd_cadparcs_parq_uc',
    'cd_identificador_cadparcs_area_protecao_apa': 'cd_cadparcs_apa_protec'
}

gdf_ucs.rename(
    columns=rename_ucs_cols,
    inplace=True
)

In [38]:
gdf_ucs.sample(2)

,id_uc,nm_uc,situacao_projeto,cat_parq_uc,cd_situacao_projeto,cd_cat_parq_uc,cd_clasfc_parq_uc,geometry,cd_cadparcs_parq_uc,nm_area,tx_tipo_categoria,tx_situacao,cd_pde_2022,cd_cadparcs_apa_protec,nm_esfera_administrativa,nm_area_protecao_ambiental
353,cadparcs_parque_unidade_conservacao.276,None,None,None,NaN,NaN,NaN,"MULTIPOLYGON (((319490.001 7405853.911, 319585...",276.0,Jaragua,Parque Estadual - UC,Existente,None,NaN,None,None
162,cadparcs_parque_unidade_conservacao.283,None,None,None,NaN,NaN,NaN,"MULTIPOLYGON (((331733.357 7404682.928, 331756...",283.0,Horto Florestal - Alberto Lofgren,Parque Estadual - UC,Existente,None,NaN,None,None
